# Tool Call LLMs
Some of the more exciting use cases for RT is when you want to cede control of the decision making process to the LLM. RT has a suite of tools that you can use to make this process much simpler. Check out an example below. 

In [1]:
from typing import Dict, Any, Set, Type
from pydantic import BaseModel, Field
import textwrap
import railtracks as rt

### Making Agents that can be used as tools:


For more information on Agents as Tools, Check out LINK TO DOCS:

In [2]:
def _critic_factory(emotion: str) -> tuple[rt.ToolManifest, str]:
    """ A factory function that creates a ToolManifest and system message for a critic tool.
    """
    system_message = f"You are a {emotion} critic of the world and you should analyze the given statement and provide a {emotion} critique of it. Be very concise and to the point."
    tool_manifest = rt.ToolManifest(
        description=f"A tool used to critique a statement {emotion}ly.",
        parameters=[rt.llm.Parameter("analysis_detail", "string", "The thing you would like to analyze")]
    )
    return system_message, tool_manifest

In [ ]:
harsh_critic_system_message, harsh_critic_tool_manifest = _critic_factory("harsh")
harsh_critic_llm = rt.llm.OpenAILLM("gpt-4o")

positive_critic_system_message, positive_critic_tool_manifest = _critic_factory("positive")
positive_critic_llm = rt.llm.AnthropicLLM("claude-3-5-sonnet-20240620")

deeper_meaning_critic_system_message, deeper_meaning_critic_tool_manifest = _critic_factory("deeper")
deeper_meaning_critic_llm = rt.llm.HuggingFaceLLM("together/deepseek-ai/DeepSeek-R1")

In [4]:
HarshCritic = rt.agent_node(
    "Harsh Critic",
    system_message=harsh_critic_system_message,
    llm_model=harsh_critic_llm,
    manifest=harsh_critic_tool_manifest,
)

PositiveCritic = rt.agent_node(
    "Positive Critic",
    system_message=positive_critic_system_message,
    llm_model=positive_critic_llm,
    manifest=positive_critic_tool_manifest,
)

DeeperMeaningCritic = rt.agent_node(
    "Deeper Meaning Critic",
    system_message=deeper_meaning_critic_system_message,
    llm_model=deeper_meaning_critic_llm,
    manifest=deeper_meaning_critic_tool_manifest,
)


### Parent Tool calling LLM that has access to agents as tools

In [5]:
class CriticOutput(BaseModel):
    harsh_critic: str = Field(description="The harsh critic of the statement")
    positive_critic: str = Field(description="The positive critic of the statement")
    deeper_meaning: str = Field(description="The deeper meaning of the statement")
    overall_score: float = Field(description="The overall score of the statement on a scale of 0 to 1")

In [6]:
system_message_critic = "You are a critic of the world and you provide comprehensive critiques of the world around you. You should utilize the provided tools to collect specific critiques and structure them before completing your answer."
Critic = rt.agent_node(
    "Critic",
    tool_nodes={HarshCritic, PositiveCritic, DeeperMeaningCritic},
    output_schema=CriticOutput,
    system_message=system_message_critic,
    llm_model=rt.llm.OpenAILLM("gpt-4o")
)

C:\Users\Aryan\dev\rc\packages\railtracks\src\railtracks\validation\node_creation\validation.py:222: UserWarning: You have set max_tool_calls to None. The llm will be able to make unlimited tool calls.
  warnings.warn(


For the parent node we will dynamically inject the model during `rt.call` 

In [7]:
template = (
    "I am writing a short story and I would like to analyze my introduction.\n"
    "\n"
    "Once upon a time there was a little boy who lived in a small village. He was a very kind and generous, but lacked an understanding"
    " of the world around him. He was always looking for ways to help others and make the world a better place. One day, he stumbled upon a"
    " magical book that would change his life forever. The book was filled with stories of adventure and mystery, and the promise of a better"
    " tomorrow."
)
with rt.Session(timeout=50):
    response = await rt.call(
        Critic, template, max_tool_calls=10
    )

Using existing session


C:\Users\Aryan\dev\rc\packages\railtracks\src\railtracks\nodes\concrete\_tool_call_base.py:74: UserWarning: You have provided max_tool_calls as a parameter and as a class variable. We will use the parameter.
  warnings.warn(
[+3.526  s] RT          : INFO     - START CREATED Critic
[+7.131  s] RT          : INFO     - Critic CREATED Positive Critic
[+7.144  s] RT          : INFO     - Critic CREATED Deeper Meaning Critic
[+7.154  s] RT          : INFO     - Critic CREATED Harsh Critic
[+10.301 s] RT          : INFO     - Positive Critic DONE
[+10.829 s] RT          : INFO     - Harsh Critic DONE
[+11.466 s] RT          : INFO     - Deeper Meaning Critic DONE
C:\Users\Aryan\dev\rc\packages\railtracks\src\railtracks\nodes\concrete\_llm_base.py:122: UserWarning: You have provided an llm model as a parameter and as a class variable. We will use the parameter.
  warnings.warn(
[+18.232 s] RT          : INFO     - Critic CREATED Structured LLM (CriticOutput)
[+24.402 s] RT          : INFO   

In [8]:
final_critic = response.structured
assert isinstance(final_critic, CriticOutput)
print(final_critic.model_dump_json(indent=2))

{
  "harsh_critic": "What a painfully saccharine and simplistic narrative setup. It's as if you tried to blend every overused trope of a children's fairy tale into one insipid package. The clichéd \"kind and generous boy\" devoid of any world understanding is such a hollow character sketch that it fails to evoke any genuine interest or connection. This magical book, a deus ex machina of narrative convenience, appears with no depth or explanation, and the vague promise of a \"better tomorrow\" is a worn-out platitude lacking any originality or insight. This story does not offer anything new or thought-provoking; it simply rehashes stale motifs without a shred of creativity to justify yet another rehashing of this tired story arc.",
  "positive_critic": "This charming narrative presents a hopeful and inspirational tale that resonates with readers of all ages. It effectively introduces a relatable protagonist - a kind-hearted boy with limited worldly experience but a strong desire to make